---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [15]:
def blight_model():
    import pandas as pd
    import numpy as np
    train=pd.read_csv('train.csv',engine='python')
    test=pd.read_csv('readonly/test.csv',engine='python')
    addresses=pd.read_csv('readonly/addresses.csv',engine='python')
    latlons=pd.read_csv('readonly/latlons.csv',engine='python')

    train.head(2)
    train[train['state'].isnull()][['ticket_id','zip_code','country']]
   

    train[train['zip_code']=='0'][['ticket_id','zip_code','state']]
    test[test['zip_code']=='0'][['ticket_id','zip_code','state']]


    
    addresses=addresses.merge(latlons,how='inner',on='address')

   
    train=train.merge(addresses,how='inner',on='ticket_id')

   
    test=test.merge(addresses,how='inner',on='ticket_id')


    train=train.dropna(axis=0,subset=['compliance'])

   
    train_simple=train.drop(labels=['inspector_name', 'violator_name',
           'violation_street_number', 'violation_street_name',
           'violation_zip_code', 'mailing_address_str_number',
           'mailing_address_str_name','non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date','disposition', 
           'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
           'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
           'payment_date', 'payment_status', 'collection_status',
           'grafitti_status', 'compliance_detail'],axis=1)
  

    test=test.drop(labels=['inspector_name', 'violator_name',
           'violation_street_number', 'violation_street_name',
           'violation_zip_code', 'mailing_address_str_number',
           'mailing_address_str_name','non_us_str_code', 'country','ticket_issued_date', 'hearing_date',
            'disposition', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
           'clean_up_cost', 'judgment_amount', 'grafitti_status'],axis=1)



    

    test.loc[test.ticket_id == 317124, 'lat'] = 42.446541
    test.loc[test.ticket_id == 317124, 'lon'] = -83.023300

    test.loc[test.ticket_id == 329689, 'lat'] = 42.358910
    test.loc[test.ticket_id == 329689, 'lon'] = -83.151329

    test.loc[test.ticket_id == 329393, 'lat'] = 42.359718
    test.loc[test.ticket_id == 329393, 'lon'] = -83.080193

    test.loc[test.ticket_id == 333990, 'lat'] = 42.376720
    test.loc[test.ticket_id == 333990, 'lon'] = -83.143211

    test.loc[test.ticket_id == 367165, 'lat'] = 42.359909
    test.loc[test.ticket_id == 367165, 'lon'] = -83.095840

    
    
    train_simple.loc[train_simple.ticket_id == 89535, 'lat'] = 42.4464515
    train_simple.loc[train_simple.ticket_id == 89535, 'lon'] = -83.0234435

    train_simple.loc[train_simple.ticket_id == 223598, 'lat'] = 42.3830098
    train_simple.loc[train_simple.ticket_id == 223598, 'lon'] = -83.0582575




    train_simple=train_simple.loc[(train_simple['agency_name']!='Neighborhood City Halls') ]


    test=test.drop(labels='state',axis=1)
    train_simple=train_simple.drop(labels='state',axis=1)

   

    train_simple=train_simple.drop(labels=['city','zip_code'],axis=1)
    test=test.drop(labels=['city','zip_code'],axis=1)

 
    train_simple1=train_simple.drop(labels=['address'],axis=1)
    test=test.drop(labels=['address'],axis=1)


    train_simple1['new_viol_code']=train_simple['violation_code'].str.replace("\-","")
    train_simple1['new_viol_code']=train_simple1['new_viol_code'].str.replace("\D","")

    

    
    test['new_viol_code']=test['violation_code'].str.replace("\-","")
    test['new_viol_code']=test['new_viol_code'].str.replace("\D","")

    train_simple1=train_simple1.drop(labels=['violation_code','violation_description'],axis=1)
    test=test.drop(labels=['violation_code','violation_description'],axis=1)

   

    from sklearn.preprocessing import OneHotEncoder

    tmp=pd.get_dummies(train_simple1['agency_name'])
    train_simple1=pd.concat([train_simple1,pd.get_dummies(train_simple1['agency_name'])], axis=1)

   
    frequency_map = (train_simple1['new_viol_code'].value_counts() / len(train_simple1) ).to_dict()
    train_simple1['new_viol_code'] = train_simple1['new_viol_code'].map(frequency_map)

    train_simple1=train_simple1[['ticket_id','Buildings, Safety Engineering & Env Department','Department of Public Works',
                                 'Detroit Police Department','Health Department','new_viol_code','lat','lon','fine_amount','compliance']]

  

    tmp1=pd.get_dummies(test['agency_name'])
    test=pd.concat([test,pd.get_dummies(test['agency_name'])], axis=1)

   
    frequency_map = (test['new_viol_code'].value_counts() / len(test) ).to_dict()
    test['new_viol_code'] = test['new_viol_code'].map(frequency_map)

    test=test[['ticket_id','Buildings, Safety Engineering & Env Department','Department of Public Works',
               'Detroit Police Department','new_viol_code','lat','lon','fine_amount']]

   
    
    train_simple1=train_simple1[['ticket_id','new_viol_code','lat','lon','fine_amount','compliance']]
    test=test[['ticket_id','new_viol_code','lat','lon','fine_amount']]


    
 
    from sklearn.cluster import KMeans
    X=train_simple1[['ticket_id','lat','lon']]
    
   

    kmeans = KMeans(n_clusters = 3, init ='k-means++')
    kmeans.fit(X[['lat','lon']]) 
    X['cluster_label'] = kmeans.fit_predict((X[['lat','lon']]))
    centers = kmeans.cluster_centers_ 
    labels = kmeans.predict(X[['lat','lon']]) 
    
  
    X=X[['ticket_id','cluster_label']]
    train_simple1=train_simple1.merge(X,how='inner',on='ticket_id')
    
   
    
    X1=test[['ticket_id','lat','lon']]
    kmeans1 = KMeans(n_clusters = 3, init ='k-means++')
    kmeans1.fit(X1[['lat','lon']]) 
    X1['cluster_label'] = kmeans1.fit_predict((X1[['lat','lon']]))
    centers1 = kmeans1.cluster_centers_ 
    labels1 = kmeans1.predict(X1[['lat','lon']]) 
    
    
  
    X1=X1[['ticket_id','cluster_label']]
    test=test.merge(X1,how='inner',on='ticket_id')
    
    

    from sklearn.tree import DecisionTreeClassifier

    X_train=train_simple1[['ticket_id','new_viol_code', 'fine_amount','cluster_label']]

    y_train=train_simple1['compliance']

    
    X_test=test[['ticket_id','new_viol_code', 'fine_amount','cluster_label']]

    dtclf = DecisionTreeClassifier(min_samples_leaf=10).fit(X_train, y_train)

    y_pred_prob=dtclf.predict_proba(X_test)
    
    res=pd.Series(data=y_pred_prob[:,1],index=test['ticket_id'])
    
    
    return res

In [16]:
blight_model()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ticket_id
284932    0.000000
285362    0.000000
285361    0.100000
285338    0.058824
285346    0.055556
285345    0.058824
285347    0.055556
285342    0.058824
285530    0.000000
284989    0.100000
285344    0.071429
285343    0.000000
285340    0.000000
285341    0.071429
285349    0.055556
285348    0.058824
284991    0.100000
285532    0.000000
285406    0.000000
285001    0.000000
285006    0.000000
285405    0.000000
285337    0.000000
285496    0.071429
285497    0.058824
285378    0.000000
285589    0.000000
285585    0.058824
285501    0.055556
285581    0.000000
            ...   
376367    0.000000
376366    0.000000
376362    0.000000
376363    0.055556
376365    0.000000
376364    0.000000
376228    0.200000
376265    0.200000
376286    0.000000
376320    0.000000
376314    0.200000
376327    0.000000
376385    0.200000
376435    0.312500
376370    0.000000
376434    0.055556
376459    0.055556
376478    0.000000
376473    0.200000
376484    0.000000
376482    0.000000
37

In [ ]:
import pandas as pd
import numpy as np
res = '{:40s}'.format('Object Type:')
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += '{:40s}'.format('Data Shape:')
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += '{:40s}'.format('Data Values Type:')
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += '{:40s}'.format('Data Values Infinity:')
res += ['Failed: values should not be infinity\n','Passed\n'][not any(np.isinf(bm))]
res += '{:40s}'.format('Data Values NaN:')
res += ['Failed: values should not be NaN\n','Passed\n'][not any(np.isnan(bm))]
res += '{:40s}'.format('Data Values in [0,1] Range:')
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += '{:40s}'.format('Data Values not all 0 or 1:')
res += ['Failed: values should be scores not predicted labels\n','Passed\n'][not all((bm.isin({0,1,0.0,1.0})))]
res += '{:40s}'.format('Index Type:')
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += '{:40s}'.format('Index Values:')
if bm.index.shape==(61001,):
    res +=['Failed: index values should match test.csv\n','Passed\n'
          ][all(pd.read_csv('test.csv',usecols=[0],index_col=0
                           ).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed: bm.index length should be 61001'
res += '{:40s}'.format('Can run model twice:')
bm2 = None
try:
    bm2 = blight_model()
    res += 'Passed\n'
except:
    res += ['Failed: second run of blight_model() threw an Exception']
res += '{:40s}'.format('Can run model twice with same results:')
if not bm2 is None:
    res += ['Failed: second run of blight_model() produced different results (this might not be a problem)\n','Passed\n'][
        all(bm.apply(lambda x:round(x,3))==bm2.apply(lambda x:round(x,3))) and all(bm.index==bm2.index)]    
print(res)